# Metodo UNESCO para calculo de salinidad

Entrada:
* Conductividad (uS/cm)
* Temperatura (T90)
* Profundidad (metros)

Salida:
* Salinidad (pps)

In [39]:
# INPUTS
temp_t90 = 13.6714
depth = 0.206
ce = 8815.365

print(f"T: {temp_t90}, D: {depth}, CE: {ce}")

T: 13.6714, D: 0.206, CE: 8815.365


In [40]:
# Transformamos la temperatura T90 a T68
temp = temp_t90 * 1.00024
print(f"T_68: {temp}")

T_68: 13.674681136


In [41]:
# Transformamos profundidad a presión en decibares
pressure = depth * 1.0197442889221
print(f"Pressure (decibar): {pressure}")

Pressure (decibar): 0.2100673235179526


In [42]:
# Transformamos a conductancia específica
specific_conductance = (ce / (1+0.02*(temp-25)))/1000
print(f"S_Conductance: {specific_conductance}")

S_Conductance: 11.396816652476923


### Empiezan los calculos de la formula

In [43]:
R = ce / 42914
R

0.20541932702614532

In [44]:
coef_Rp = {
    "e1": 0.0000207,
    "e2": -0.000000000637,
    "e3": 3.989E-15,
    "d1": 0.03426,
    "d2": 0.0004464,
    "d3": 0.4215,
    "d4": -0.003107
}
coef_Rp

{'e1': 2.07e-05,
 'e2': -6.37e-10,
 'e3': 3.989e-15,
 'd1': 0.03426,
 'd2': 0.0004464,
 'd3': 0.4215,
 'd4': -0.003107}

In [45]:
Rp = 1 + pressure * (coef_Rp["e1"] + (coef_Rp["e2"] * pressure) + (coef_Rp["e3"]*pressure**2))/(1 + coef_Rp["d1"]*temp + coef_Rp["d2"]*temp**2 + (coef_Rp["d3"]*coef_Rp["d4"])*R)
Rp

1.0000028023218088

In [46]:
coef_rt = {
    "c0": 0.6766097,
    "c1": 0.0200664,
    "c2": 0.0001104259,
    "c3": -0.00000069698,
    "c4": 0.0000000010031
}
coef_rt

{'c0': 0.6766097,
 'c1': 0.0200664,
 'c2': 0.0001104259,
 'c3': -6.9698e-07,
 'c4': 1.0031e-09}

In [47]:
rt = coef_rt["c0"] + coef_rt["c1"]*temp + coef_rt["c2"]*temp**2 + coef_rt["c3"]*temp**3 + coef_rt["c4"]*temp**4
rt

0.9699134356152391

In [48]:
Rt = R / (Rp*rt)
Rt

0.21179080919360233

In [49]:
coef_AS = {
    "b0": 0.0005,
    "b1": -0.0056,
    "b2": -0.0066,
    "b3": -0.0375,
    "b4": 0.0636,
    "b5": -0.0144,
    "k": 0.0162
}
coef_AS

{'b0': 0.0005,
 'b1': -0.0056,
 'b2': -0.0066,
 'b3': -0.0375,
 'b4': 0.0636,
 'b5': -0.0144,
 'k': 0.0162}

In [50]:
AS = (temp-15)/(1+(coef_AS["k"]*(temp-15))) * (coef_AS["b0"]+coef_AS["b1"]*Rt**0.5 + coef_AS["b2"]*Rt+coef_AS["b3"]*Rt**1.5 + coef_AS["b4"]*Rt**2 + coef_AS["b5"]*Rt**2.5)
AS

0.0061956592008433

In [51]:
coef_S = {
    "a0": 0.008,
    "a1": -0.1692,
    "a2": 25.3851,
    "a3": 14.0941,
    "a4": -7.0261,
    "a5": 2.7081
}
coef_S

{'a0': 0.008,
 'a1': -0.1692,
 'a2': 25.3851,
 'a3': 14.0941,
 'a4': -7.0261,
 'a5': 2.7081}

In [52]:
S = coef_S["a0"] + coef_S["a1"]*Rt**0.5 + coef_S["a2"]*Rt + coef_S["a3"]*Rt**1.5 + coef_S["a4"]*Rt**2 + coef_S["a5"]*Rt**2.5 + AS
S

6.427123294175525

### Agrupamos en una función

In [53]:
def get_salinity(temp_t90, depth, ce_uScm):
    # Transform inputs:
    temp = temp_t90 * 1.00024
    pressure = depth * 1.0197442889221
    specific_conductance = (ce_uScm / (1+0.02*(temp-25)))/1000

    # Chapter 3. Determination of salinity [Methods of Seawater Analysis]
    R = ce_uScm / 42914
    Rp = 1 + pressure * (coef_Rp["e1"] + (coef_Rp["e2"] * pressure) + (coef_Rp["e3"]*pressure**2))/(1 + coef_Rp["d1"]*temp + coef_Rp["d2"]*temp**2 + (coef_Rp["d3"]*coef_Rp["d4"])*R)
    rt = coef_rt["c0"] + coef_rt["c1"]*temp + coef_rt["c2"]*temp**2 + coef_rt["c3"]*temp**3 + coef_rt["c4"]*temp**4
    Rt = R / (Rp*rt)
    AS = (temp-15)/(1+(coef_AS["k"]*(temp-15))) * (coef_AS["b0"]+coef_AS["b1"]*Rt**0.5 + coef_AS["b2"]*Rt+coef_AS["b3"]*Rt**1.5 + coef_AS["b4"]*Rt**2 + coef_AS["b5"]*Rt**2.5)
    S = coef_S["a0"] + coef_S["a1"]*Rt**0.5 + coef_S["a2"]*Rt + coef_S["a3"]*Rt**1.5 + coef_S["a4"]*Rt**2 + coef_S["a5"]*Rt**2.5 + AS

    # Salinity (pps)
    return S

    

In [54]:
get_salinity(temp_t90=13.6714, depth=0.206, ce_uScm=8815.365)

6.427123294175525